In [4]:
import json
import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save as alt_save

In [15]:
metmap_tissues = snakemake.params["metmap_tissues"]
tm_to_metmap = snakemake.params["tm_to_metmap"]

In [16]:
df = pd.DataFrame(columns=["tissue", "fold", "metric", "value"])

for fold_i, fold_input in enumerate(snakemake.input):
    with open(fold_input) as f:
        fold_results = json.load(f)
    for tissue in metmap_tissues:
        fold_tissue_results = fold_results[tissue]
        for metric, value in fold_tissue_results.items():
            df = df.append({
                "tissue": tm_to_metmap[tissue],
                "fold": fold_i,
                "metric": metric,
                "value": value
            }, ignore_index=True)

In [18]:
plot = alt.Chart(df).mark_boxplot().encode(
    x=alt.X("tissue:N"),
    y=alt.Y("value:Q"),
    column=alt.Column("metric:N")
).resolve_scale(y='independent')

plot

In [19]:
alt_save(plot, snakemake.output["plot"])